In [19]:
!pip install torch
!pip install torchaudio
!pip install torch-geometric
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install matplotlib
!pip install kagglehub

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
import torchaudio
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
import torch.nn.functional as F
from sklearn.metrics import classification_report
import kagglehub
from pathlib import Path
import os

# Data Loading and Processing

In [21]:
path = kagglehub.dataset_download("murtadhanajim/gender-recognition-by-voiceoriginal")

100%|██████████| 1.31G/1.31G [00:19<00:00, 73.9MB/s]

Extracting files...


In [44]:
data_paths = list(Path(path).rglob("*.wav"))
print(len(data_paths))
male_files = list(Path(path, "data/male").rglob("*.wav"))
female_files = list(Path(path, "data/female").rglob("*.wav"))
print(len(male_files), len(female_files))

16148
10380 5768


In [43]:
def PreprocessAudio(data, target_sample_rate=32000):
    audio_files = []
    for sample in data:
        waveform, sample_rate = torchaudio.load(sample)
        # Resample
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(sample_rate, target_sample_rate)
            waveform = resampler(waveform)
        # Normalize
        max_amp = torch.max(torch.abs(waveform))
        if max_amp > 0:
            waveform = waveform / max_amp
        audio_files.append((sample, waveform))
    return audio_files

In [45]:
data = PreprocessAudio(data_paths)

In [51]:
# data is stored as (path, waveform)
print(data[0])

(PosixPath('/root/.cache/kagglehub/datasets/murtadhanajim/gender-recognition-by-voiceoriginal/versions/1/data/female/arctic_a0260(7).wav'), tensor([[0.0043, 0.0049, 0.0043,  ..., 0.0000, 0.0000, 0.0000]]))


# Audio Signal Implementaion

# Graph Construction

# GNN Architectures Implementation

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        return self.conv2(x, edge_index)

In [52]:
class GYATT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=4, concat=True)
        self.conv2 = GATConv(hidden_dim * 4, output_dim, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        return self.conv2(x, edge_index)

# Performance Evaluation